<a href="https://colab.research.google.com/github/dongxiangtranslationproject/dongxiangtranslationproject.github.io/blob/main/Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Sentence Import and Formatting

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install sacrebleu

In [ ]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

device = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_DIR1 = "/content/drive/MyDrive/my_nllb_DC_model"
tokenizer1 = AutoTokenizer.from_pretrained(MODEL_DIR1)
model1 = AutoModelForSeq2SeqLM.from_pretrained(MODEL_DIR1).to(device)
model1.eval()

def translate1(text, src_lang="sce_Latn", tgt_lang="zho_Hans",
               a=16, b=1.5, max_input_length=1024, **kwargs):
    tokenizer1.src_lang = src_lang
    inputs = tokenizer1(text, return_tensors="pt", padding=True,
                        truncation=True, max_length=max_input_length).to(model1.device)
    result = model1.generate(
        **inputs,
        forced_bos_token_id=tokenizer1.convert_tokens_to_ids(tgt_lang),
        max_new_tokens=int(a + b * inputs.input_ids.shape[1]),
        **kwargs
    )
    outputs = tokenizer1.batch_decode(result, skip_special_tokens=True)
    return outputs

In [ ]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

device = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_DIR2 = "/content/drive/MyDrive/my_nllb_DCword_model"
tokenizer2 = AutoTokenizer.from_pretrained(MODEL_DIR2)
model2 = AutoModelForSeq2SeqLM.from_pretrained(MODEL_DIR2).to(device)
model2.eval()

def translate2(text, src_lang="sce_Latn", tgt_lang="zho_Hans",
               a=16, b=1.5, max_input_length=1024, **kwargs):
    tokenizer2.src_lang = src_lang
    inputs = tokenizer2(text, return_tensors="pt", padding=True,
                        truncation=True, max_length=max_input_length).to(model2.device)
    result = model2.generate(
        **inputs,
        forced_bos_token_id=tokenizer2.convert_tokens_to_ids(tgt_lang),
        max_new_tokens=int(a + b * inputs.input_ids.shape[1]),
        **kwargs
    )
    outputs = tokenizer2.batch_decode(result, skip_special_tokens=True)
    return outputs

In [ ]:
buffer = input("please enter a Dongxiang sentence:")
if buffer == "":
  text = " Chi eqiegvan gaodase"
else:
  text = buffer
print("Source:", text)
print(f"word-level model Translation: {translate2(text,no_repeat_ngram_size=3, num_beams=5)}\n"
      f"sentence-level model Translation: {translate1(text,no_repeat_ngram_size=3, num_beams=5)}")

please enter a Dongxiang sentence:
Source:  Chi eqiegvan gaodase
word-level model Translation: ['你早点好']
sentence-level model Translation: ['愿你早日康复']


Template: <br>
Dongxiang: Chi eqiegvan gaodase <br>
Chinese: 愿你早日康复

In [ ]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

device = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_DIR3 = "/content/drive/MyDrive/my_nllb_CD_model"
tokenizer3 = AutoTokenizer.from_pretrained(MODEL_DIR3)
model3 = AutoModelForSeq2SeqLM.from_pretrained(MODEL_DIR3).to(device)
model3.eval()

def translate3(text, src_lang="zho_Hans", tgt_lang="sce_Latn",
               a=16, b=1.5, max_input_length=1024, **kwargs):
    tokenizer3.src_lang = src_lang
    inputs = tokenizer3(text, return_tensors="pt", padding=True,
                        truncation=True, max_length=max_input_length).to(model1.device)
    result = model3.generate(
        **inputs,
        forced_bos_token_id=tokenizer3.convert_tokens_to_ids(tgt_lang),
        max_new_tokens=int(a + b * inputs.input_ids.shape[1]),
        **kwargs
    )
    outputs = tokenizer3.batch_decode(result, skip_special_tokens=True)
    return outputs

In [ ]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

device = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_DIR4 = "/content/drive/MyDrive/my_nllb_CDword_model"
tokenizer4 = AutoTokenizer.from_pretrained(MODEL_DIR4)
model4 = AutoModelForSeq2SeqLM.from_pretrained(MODEL_DIR4).to(device)
model4.eval()

def translate4(text, src_lang="zho_Hans", tgt_lang="sce_Latn",
               a=16, b=1.5, max_input_length=1024, **kwargs):
    tokenizer4.src_lang = src_lang
    inputs = tokenizer4(text, return_tensors="pt", padding=True,
                        truncation=True, max_length=max_input_length).to(model1.device)
    result = model4.generate(
        **inputs,
        forced_bos_token_id=tokenizer4.convert_tokens_to_ids(tgt_lang),
        max_new_tokens=int(a + b * inputs.input_ids.shape[1]),
        **kwargs
    )
    outputs = tokenizer4.batch_decode(result, skip_special_tokens=True)
    return outputs

In [ ]:
buffer = input("please enter a Chinese sentence:")
if buffer == "":
  text = ["愿你早日康复", "愿你早日康复"]
else:
  text = buffer
print("Source:", text)
print(f"word-level model Translation: {translate4(text,no_repeat_ngram_size=3, num_beams=5)}\n"
      f"sentence-level model Translation: {translate3(text,no_repeat_ngram_size=3, num_beams=5)}")

please enter a Chinese sentence:我要买新车
Source: 我要买新车
word-level model Translation: ['bi shinche agine']
sentence-level model Translation: ['bi shini chezi agine giezho']


In [ ]:
def eval_bleu_chrf(hyps, refs, tgt_is_zh=False):
    bleu_tok = "zh" if tgt_is_zh else "13a"
    bleu = sacrebleu.BLEU(tokenize=bleu_tok)
    chrf = sacrebleu.CHRF(word_order=2)

    bleu_score = bleu.corpus_score(hyps, [refs])
    chrf_score = chrf.corpus_score(hyps, [refs])
    return bleu_score, chrf_score

In [ ]:
dongxiang = ["hhe enedu cha ochi",
             "mayi chezi agi.",
             "ta ene olon cha izhiewo ye",
             "magvashijie budan izhiene.",
             "chi enedu kala echiwo?.",
             "chi shi ali oron kun?.",

             "chi khala giemere uzhele echiwo?.",
             "ene shi mini ochin zhiao.",
             "Hhe morei unuzhi naduzho.",
             "Tegha gatashi qianguzhi ijiene.",
             "Khara naran doura shu uzhese nudunde gao wiwo."
             ]

In [ ]:
Chinese = ["他今天喝茶",
           "我们买个车子。",
           "你们最近吃了吗",
           "明天吃饭。",
           "你今天去哪了？",
           "你是哪里人？",

           "你去哪看病了？",
           "这是我的妹妹。",
           "他骑马玩。",
           "鸡捡小石子吃。",
           "烈日下看书的话对眼睛不好。"
           ]

We randomly selected 11 sentences to compute BLEU and ChrF scores. In addition to the two “sentence-level models,” which include artificially generated bilingual sentences, we also trained two “word-level models” that take bilingual word pairs as input instead of full sentences.

In [ ]:
src_dx = dongxiang
ref_zh = Chinese

hyps_zh = translate1(src_dx, src_lang="sou_Latn", tgt_lang="zho_Hans",no_repeat_ngram_size=3, num_beams=5)
bleu_zh, chrf_zh = eval_bleu_chrf(hyps_zh, ref_zh, tgt_is_zh=True)
print("Sentence[DX→ZH] BLEU:", bleu_zh)
print("Sentence[DX→ZH] ChrF++:", chrf_zh)

Sentence[DX→ZH] BLEU: BLEU = 44.00 70.5/50.7/39.3/26.7 (BP = 1.000 ratio = 1.026 hyp_len = 78 ref_len = 76)
Sentence[DX→ZH] ChrF++: chrF2++ = 34.37


In [ ]:
src_dx = dongxiang
ref_zh = Chinese

hyps_zh = translate2(src_dx, src_lang="sou_Latn", tgt_lang="zho_Hans",no_repeat_ngram_size=3, num_beams=5)
bleu_zh, chrf_zh = eval_bleu_chrf(hyps_zh, ref_zh, tgt_is_zh=True)
print("Word[DX→ZH] BLEU:", bleu_zh)
print("Word[DX→ZH] ChrF++:", chrf_zh)

Word[DX→ZH] BLEU: BLEU = 24.69 50.0/28.4/19.6/13.3 (BP = 1.000 ratio = 1.026 hyp_len = 78 ref_len = 76)
Word[DX→ZH] ChrF++: chrF2++ = 20.69


In [ ]:
src_dx = Chinese
ref_zh = dongxiang

hyps_zh = translate3(src_dx, src_lang="zho_Hans", tgt_lang="sou_Latn",no_repeat_ngram_size=3, num_beams=5)
bleu_zh, chrf_zh = eval_bleu_chrf(hyps_zh, ref_zh, tgt_is_zh=True)
print("Sentence[ZH→DX] BLEU:", bleu_zh)
print("Sentence[ZH→DX] ChrF++:", chrf_zh)

Sentence[ZH→DX] BLEU: BLEU = 46.23 65.1/46.2/41.5/36.7 (BP = 1.000 ratio = 1.000 hyp_len = 63 ref_len = 63)
Sentence[ZH→DX] ChrF++: chrF2++ = 59.80


In [ ]:
src_dx = Chinese
ref_zh = dongxiang

hyps_zh = translate4(src_dx, src_lang="zho_Hans", tgt_lang="sou_Latn",no_repeat_ngram_size=3, num_beams=5)
bleu_zh, chrf_zh = eval_bleu_chrf(hyps_zh, ref_zh, tgt_is_zh=True)
print("Word[ZH→DX] BLEU:", bleu_zh)
print("Word[ZH→DX] ChrF++:", chrf_zh)

Word[ZH→DX] BLEU: BLEU = 8.97 46.0/17.3/4.9/1.7 (BP = 1.000 ratio = 1.000 hyp_len = 63 ref_len = 63)
Word[ZH→DX] ChrF++: chrF2++ = 36.05
